<a href="https://colab.research.google.com/github/typijo/bert-tltc/blob/master/finetune_pretrained_%E6%97%A5%E6%9C%AC%E8%AA%9EBERT_using_statutory_sentence_(tpu_version).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You need your GCP storage bucket to be set as follows:
```
/(backet name)/
  pretrain_statutory_sentence/
    (training data).txt # training data (see below)
    vocab.txt # vocabulary (one line one word)
  bert/
    Japanese_L-12_H-768_A-12_E-30_BPE/
      bert_model.ckpt.index
      bert_model.ckpt.meta
      bert_model.ckpt.data-00000-of-00001	
      bert_config.json
```

Format of the training data
- one line one sentence
- documents are separated by one blank line

**example**
```
doc1-line1-word1 doc1-line1-word2 ...
doc1-line2-word1 doc1-line2-word2 ...
...

doc2-line1-word1 doc2-line1-word2 ...
doc2-line2-word1 doc2-line2-word2 ...
...
```

In [0]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

from google.colab import auth
auth.authenticate_user()

tf.logging.set_verbosity(tf.logging.INFO)
tf.app.flags.DEFINE_string('f', '', 'kernel')

tf.test.gpu_device_name()

W0801 03:43:12.355468 139677679544192 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



''

In [0]:
# load BERT codes
import sys

!test -d bert-tltc || git clone https://github.com/typijo/bert-tltc.git #changed to my repo
if not 'bert-tltc' in sys.path:
  sys.path += ['bert-tltc']

Cloning into 'bert-tltc'...
remote: Enumerating objects: 553, done.
remote: Total 553 (delta 0), reused 0 (delta 0), pack-reused 553
Receiving objects: 100% (553/553), 317.33 KiB | 4.02 MiB/s, done.
Resolving deltas: 100% (338/338), done.


### tfrecordの作成

parameters are inherited from 日本語BERT and original bert
https://github.com/google-research/bert#pre-training-with-bert

- vocab is from 日本語BERT
- inherition exception: dupe_factor here is 3 not 5 that is the original one

In [0]:
# your bucket where the textfile is
BUCKET = "japan-legal-term-correction" #@param {type:"string"}

# textfile
TEXTFILE_NAME = "corpus_elaws_1500.txt" #@param {type:"string"}

# suffix for management
SUFFIX_TFRECORD = "1500" #@param {type:"string"}

DATASET_PATH = "gs://{}/pretrain_statutory_sentence".format(BUCKET)

TEXT_FILE = "{}/{}".format(DATASET_PATH, TEXTFILE_NAME)
TFRECORD_FILE = "{}/tf_examples{}.tfrecord".format(DATASET_PATH, SUFFIX_TFRECORD)
VOCAB_FILE = "{}/vocab.txt".format(DATASET_PATH)

#### 訓練データの作成

In [0]:
%env TEXT_FILE={TEXT_FILE}
%env TFRECORD_FILE={TFRECORD_FILE}
%env VOCAB_FILE={VOCAB_FILE}

env: TEXT_FILE=gs://japan-legal-term-correction/pretrain_statutory_sentence/corpus_elaws_1500.txt
env: TFRECORD_FILE=gs://japan-legal-term-correction/pretrain_statutory_sentence/tf_examples1500.tfrecord
env: VOCAB_FILE=gs://japan-legal-term-correction/pretrain_statutory_sentence/vocab.txt


In [0]:
%%bash
python bert-tltc/create_pretraining_data.py \
--input_file=${TEXT_FILE} --output_file=${TFRECORD_FILE} --vocab_file=${VOCAB_FILE} \
--do_lower_case=False --tokenize_chinese_chars=False --max_seq_length=128 --max_predictions_per_seq=20 --masked_lm_prob=0.15 --random_seed=0 --dupe_factor=3 2>&1

W0801 03:45:05.082638 139981399807872 deprecation_wrapper.py:119] From bert-tltc/create_pretraining_data.py:446: The name tf.app.run is deprecated. Please use tf.compat.v1.app.run instead.

W0801 03:45:05.083896 139981399807872 deprecation_wrapper.py:119] From bert-tltc/create_pretraining_data.py:414: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.

W0801 03:45:05.084145 139981399807872 deprecation_wrapper.py:119] From bert-tltc/create_pretraining_data.py:414: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.

W0801 03:45:05.084367 139981399807872 deprecation_wrapper.py:119] From /content/bert-tltc/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.

W0801 03:45:06.985617 139981399807872 deprecation_wrapper.py:119] From bert-tltc/create_pretraining_data.py:421: The name tf.gfile.Glob is deprecated. Please use tf.io.gfile.glob instead.

W0801 03:45:07

#### pretraining関係の定数

In [0]:
MODEL_PATH = "{}/model".format(DATASET_PATH)

JPNBERT_RELPATH = "bert/Japanese_L-12_H-768_A-12_E-30_BPE" # @param {"type": "string"}
JPNBERT_PATH = "gs://{}/{}".format(BUCKET, JPNBERT_RELPATH)

BERT_CONFIG_FILE = "{}/bert_config.json".format(JPNBERT_PATH)
INIT_CP = "{}/bert_model.ckpt".format(JPNBERT_PATH)

USE_TPU = True

SAVE_CHECKPOINTS_STEPS = 20000
ITERATIONS_PER_LOOP = 1000
NUM_TPU_CORES = 8

LEARNING_RATE = 2e-5
NUM_TRAIN_STEPS = 100000
NUM_WARMUP_STEPS = 1000
NUM_EVAL_STEPS = 100

TRAIN_BATCH_SIZE = 32 #512
EVAL_BATCH_SIZE = 512 #8
PRED_BATCH_SIZE = 64 #8

# params that are defined in config file
MAX_SEQ_LENGTH = 128
MAX_PREDICTIONS_PER_SEQ = 20


import run_pretraining
import modeling

### finetuning

parameters are inherited from 日本語BERT and original bert
https://github.com/google-research/bert#pre-training-with-bert

In [0]:
def make_estimator(init_checkpoint=INIT_CP, model_dir=MODEL_PATH):
  bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG_FILE)
  
  tpu_cluster_resolver = None
  if USE_TPU and TPU_ADDRESS:
    tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)

  is_per_host = tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2
  run_config = tf.contrib.tpu.RunConfig(
      cluster=tpu_cluster_resolver,
      model_dir=model_dir,
      save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
      keep_checkpoint_max=10,
      tpu_config=tf.contrib.tpu.TPUConfig(
          iterations_per_loop=ITERATIONS_PER_LOOP,
          num_shards=NUM_TPU_CORES,
          per_host_input_for_training=is_per_host))

  model_fn = run_pretraining.model_fn_builder(
      bert_config=bert_config,
      init_checkpoint=init_checkpoint,
      learning_rate=LEARNING_RATE,
      num_train_steps=NUM_TRAIN_STEPS,
      num_warmup_steps=NUM_WARMUP_STEPS,
      use_tpu=USE_TPU,
      use_one_hot_embeddings=USE_TPU)
  
  # If TPU is not available, this will fall back to normal Estimator on CPU
  # or GPU.
  estimator = tf.contrib.tpu.TPUEstimator(
      use_tpu=USE_TPU,
      model_fn=model_fn,
      config=run_config,
      train_batch_size=TRAIN_BATCH_SIZE,
      eval_batch_size=EVAL_BATCH_SIZE,
      predict_batch_size=PRED_BATCH_SIZE)
  
  return estimator

In [0]:
# exec this if you use TPU
if USE_TPU:
  assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
  TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print('TPU address is', TPU_ADDRESS)

  with tf.Session(TPU_ADDRESS) as session:
    print('TPU devices:')
    pprint.pprint(session.list_devices())

    # Upload credentials to TPU.
    with open('/content/adc.json', 'r') as f:
      auth_info = json.load(f)
    tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
    # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.60.154.58:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 4580005805586410807),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 17394964189653461345),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 2257332226448154062),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 4768680384725283491),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 18253737298473624293),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 10979287439995388824),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 3790691428889049938),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 7594210683778320102),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 4874217104154

In [0]:
estimator = None
def pretrain(do_train=True, do_eval=True):
  global estimator

  tf.gfile.MakeDirs(MODEL_PATH)

  input_files = []
  for input_pattern in TFRECORD_FILE.split(","):
    input_files.extend(tf.gfile.Glob(input_pattern))

  tf.logging.info("*** Input Files ***")
  for input_file in input_files:
    tf.logging.info("  %s" % input_file)

  estimator = make_estimator()

  if do_train:
    tf.logging.info("***** Running training *****")
    tf.logging.info("  Batch size = %d", TRAIN_BATCH_SIZE)
    train_input_fn = run_pretraining.input_fn_builder(
        input_files=input_files,
        max_seq_length=MAX_SEQ_LENGTH,
        max_predictions_per_seq=MAX_PREDICTIONS_PER_SEQ,
        is_training=True)
    estimator.train(input_fn=train_input_fn, max_steps=NUM_TRAIN_STEPS)

  if do_eval:
    tf.logging.info("***** Running evaluation *****")
    tf.logging.info("  Batch size = %d", EVAL_BATCH_SIZE)

    eval_input_fn = run_pretraining.input_fn_builder(
        input_files=input_files,
        max_seq_length=MAX_SEQ_LENGTH,
        max_predictions_per_seq=MAX_PREDICTIONS_PER_SEQ,
        is_training=False)

    result = estimator.evaluate(
        input_fn=eval_input_fn, steps=NUM_EVAL_STEPS)

    output_eval_file = os.path.join(MODEL_PATH, "eval_results.txt")
    with tf.gfile.GFile(output_eval_file, "w") as writer:
      tf.logging.info("***** Eval results *****")
      for key in sorted(result.keys()):
        tf.logging.info("  %s = %s", key, str(result[key]))
        writer.write("%s = %s\n" % (key, str(result[key])))

In [0]:
tf.logging.set_verbosity(tf.logging.WARN)
pretrain(do_train=True, do_eval=False)

W0801 03:58:33.414719 139677679544192 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f08f46c2b70>) includes params argument, but params are not passed to Estimator.
W0801 03:58:33.852328 139677679544192 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/training_util.py:236: Variable.initialized_value (from tensorflow.python.ops.variables) is deprecated and will be removed in a future version.
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
W0801 03:58:33.876772 139677679544192 deprecation_wrapper.py:119] From bert-tltc/run_pretraining.py:347: The name tf.FixedLenFeature is deprecated. Please use tf.io.FixedLenFeature instead.

W0801 03:58:33.889818 139677679544192 deprecation.py:323] From bert-tltc/run_pretraining.py:378: parallel_interleave (from tensorflow.contrib.data.python.ops.interleave_ops) is

### prediction

In [0]:
CLIST_FILE = "{}/common_format5_statutory_27_unite.json".format(DATASET_PATH)
TFRECORD_PRED_FILE = "{}/tf_examples_pred.tfrecord".format(DATASET_PATH)
ANSWER_FILE = "{}/answers.txt".format(DATASET_PATH)

# load dataset
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
ADDR_DATASET_BASE = "/content/gdrive/My Drive/colab_data/"
NAME_DATASET =  'bert_unite_global_train-eval-test_jpnbert/{}' #@param {type:"string"}
TASK_DATA_DIR = ADDR_DATASET_BASE + NAME_DATASET
TASK_DATA_FILE = ADDR_DATASET_BASE + NAME_DATASET + "/{}.tsv"

def get_name_resultfile(prefix):
  return DATASET_PATH + "/result_{}".format(prefix)

def set_name_dataset(dataset):
  global NAME_DATASET, TASK_DATA_DIR
  NAME_DATASET = dataset
  TASK_DATA_DIR = ADDR_DATASET_BASE + NAME_DATASET
set_name_dataset(NAME_DATASET)

def get_addr_dataset(termsetid, datatype="train"):
  if datatype == "test":
    num = 0
  elif datatype == "train":
    num = 1
  elif datatype == "eval":
    num = 2
  else:
    num = 0
  
  ret = TASK_DATA_FILE.format(termsetid, num)
  return ret

def get_addr_datasetdir(termsetid):
  ret = TASK_DATA_DIR.format(termsetid)
  return ret


# use to delete tf.flags to avoid collision
def delete_tfflags(keys):
  for key in keys:
    if hasattr(tf.flags.FLAGS, key):
      tf.flags.FLAGS.__delattr__(key)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import json

import tokenization

class ChoiceMaker:
  """
  Offer function f: gtid -> (wid_answer, [wids_sametermset])
  """
  
  def __init__(self, vocab_file=VOCAB_FILE, clist_file=CLIST_FILE):
    self.WID_NONE = -1
    self.vocab = tokenization.load_vocab(vocab_file)
    self.inv_vocab = {v: k for k, v in self.vocab.items()}
    self.clist = ChoiceMaker.load_clist(clist_file)
    
    self.gtid2term = []
    self.gtid2wid = []
    self.gtid2lid = []
    self.lid2wids = []
    self.lid2gtids = []
    
    for i, terms in enumerate(self.clist):
      self.lid2wids += [[]]
      self.lid2gtids += [[]]
      
      for term in terms["terms"]:
        term_unite = " ".join(term)
        
        self.gtid2term += [term_unite]
        self.gtid2lid += [i]
        
        wid = self.get_wid(term_unite)
        
        self.gtid2wid += [wid]
        
        if wid != self.WID_NONE:
          self.lid2wids[-1] += [wid]
          self.lid2gtids[-1] += [len(self.gtid2term)-1]
  
  @classmethod
  def load_clist(cls, path):
    with tf.gfile.GFile(path, "r") as f:
      clist_tmp = json.load(f)["0"]

    clist = []
    terms_str_exist = set()
    for term_from in clist_tmp:
        terms_str = "__and__".join(list(sorted([" ".join(term) for term in clist_tmp[term_from]["terms"]])))
        if terms_str not in terms_str_exist:
            clist += [{"terms": terms_str, "comment": clist_tmp[term_from]["comment"]}]
            terms_str_exist.add(terms_str)
    for terms in clist:
        terms_str = terms["terms"]
        terms["terms"] = [term.split(" ") for term in terms_str.split("__and__")]

    return clist
  
  def get_wid(self, term):
    if term in self.vocab:
      return self.vocab[term]
    else:
      return self.WID_NONE
  
  def get_choices_from_gtid(self, gtid):
    lid = self.gtid2lid[gtid]
    return self.lid2wids[lid]

cm = ChoiceMaker()

W0729 09:48:38.520940 139768208836480 deprecation_wrapper.py:119] From bert-tltc/tokenization.py:125: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.



In [0]:
from collections import OrderedDict

delete_tfflags([
    "vocab_file", "output_dir", "init_checkpoint", "max_seq_length", "do_train", "do_eval", 
    "train_batch_size", "eval_batch_size", "learning_rate", "save_checkpoints_steps",
    "iterations_per_loop", "use_tpu", "tpu_name", "tpu_zone", "gcp_project", 
    "num_tpu_cores", "input_file", "output_dir", "max_predictions_per_seq"
])
from create_pretraining_data import create_int_feature, create_float_feature

answers = []
localqid2globalqid = []
def write_masklm_tfrecord_from_classifier_features(features, dst=TFRECORD_PRED_FILE):
  writer = tf.python_io.TFRecordWriter(dst)
    
  for i, feature in enumerate(features): # feature is an instance of InputFeatures
    # 必要なフィールドを追加
    gtid = int(feature.label_id)
    wid_answer = cm.gtid2wid[gtid]
    if wid_answer == cm.WID_NONE:
      continue

    try:
      feature.masked_lm_positions = [feature.input_ids.index(cm.vocab["[MASK]"])]
    except ValueError:
      continue
    
    feature.masked_lm_ids = [wid_answer]
    feature.masked_lm_weights = [1.0]

    while len(feature.masked_lm_positions) < MAX_PREDICTIONS_PER_SEQ:
      feature.masked_lm_positions.append(0)
      feature.masked_lm_ids.append(0)
      feature.masked_lm_weights.append(0.0)


    # tfrecordに変換
    feature_tfr = OrderedDict()
    feature_tfr["input_ids"] = create_int_feature(feature.input_ids)
    feature_tfr["input_mask"] = create_int_feature(feature.input_mask)
    feature_tfr["segment_ids"] = create_int_feature(feature.segment_ids)
    feature_tfr["masked_lm_positions"] = create_int_feature(feature.masked_lm_positions)
    feature_tfr["masked_lm_ids"] = create_int_feature(feature.masked_lm_ids)
    feature_tfr["masked_lm_weights"] = create_float_feature(feature.masked_lm_weights)
    feature_tfr["next_sentence_labels"] = create_int_feature([0])

    tf_example = tf.train.Example(features=tf.train.Features(feature=feature_tfr))
    writer.write(tf_example.SerializeToString())
    
    answers.append(gtid)
    localqid2globalqid.append(i)
  
  print(len(answers), "examples written")
  
  writer.close()
  
  with tf.gfile.GFile(ANSWER_FILE, "w") as f:
    f.write(",".join([str(i) + "-" + str(j) for i, j in zip(answers, localqid2globalqid)]) + "\n")

In [0]:
delete_tfflags([
    "vocab_file", "output_dir", "init_checkpoint", "max_seq_length", "do_train", "do_eval", 
    "train_batch_size", "eval_batch_size", "learning_rate", "save_checkpoints_steps",
    "iterations_per_loop", "use_tpu", "tpu_name", "tpu_zone", "gcp_project", 
    "num_tpu_cores", "input_file", "output_dir", "max_predictions_per_seq",
    "bert_config_file", "do_lower_case", "master"
])
import run_classifier

processor = None
def make_tfrecord_eval(dst=TFRECORD_PRED_FILE):
  global processor
  processor = run_classifier.TltcProcessorWithGlobalTid()
  
  exists = !gsutil -q stat $TFRECORD_PRED_FILE; echo $?
  if int(exists[0]) == 0:
    print("eval tfrecord already exists. read it, with answer_file")
    
    global answers, localqid2globalqid
    answers = []
    localqid2globalqid = []
    with tf.gfile.GFile(ANSWER_FILE, "r") as f:
      for expr in f.read().strip().split(","):
        tokens = expr.split("-")
        answers.append(int(tokens[0]))
        localqid2globalqid.append(int(tokens[1]))
    
    return
  else:
    print("no eval tfrecord. make now")
  
  tokenizer = tokenization.FullTokenizer(vocab_file=VOCAB_FILE, do_lower_case=False, do_tokenize_chinese_chars=False, metawords=["[MASK]", "[UNK]"]) 
  
  pred_examples = processor.get_dev_examples_cv(get_addr_datasetdir(0), 0)
  pred_features = run_classifier.convert_examples_to_features(pred_examples, label_list=processor.get_labels(), max_seq_length=MAX_SEQ_LENGTH, tokenizer=tokenizer)
  write_masklm_tfrecord_from_classifier_features(pred_features)

In [0]:
import numpy as np
def predict(prefix_resultfile="test", checkpoint_path=None):
  make_tfrecord_eval()
  
  print('***** Started prediction at {} *****'.format(datetime.datetime.now()))
  
  pred_input_fn = run_pretraining.input_fn_builder(
        input_files=TFRECORD_PRED_FILE,
        max_seq_length=MAX_SEQ_LENGTH,
        max_predictions_per_seq=MAX_PREDICTIONS_PER_SEQ,
        is_training=False,
        is_predicting=True)  
  results = estimator.predict(input_fn=pred_input_fn, checkpoint_path=checkpoint_path)
  
  print('***** Finished prediction at {} *****'.format(datetime.datetime.now()))
  
  label_list = processor.get_labels()
  
  confmat = [[0 for _ in range(len(label_list))] for _ in range(len(label_list))]
  counts = [0 for _ in range(len(label_list))]
  
  str_for_writer = ""
  for i, result in enumerate(results):
    if i % MAX_PREDICTIONS_PER_SEQ != 0:
      continue
    
    if i//MAX_PREDICTIONS_PER_SEQ >= len(answers):
      print("MAX_PRED_PASSED")
      break
      
    ans = answers[i//MAX_PREDICTIONS_PER_SEQ]
    options = cm.get_choices_from_gtid(ans)
    
    probs = result["probabilities"][options]
    pred = cm.lid2gtids[cm.gtid2lid[ans]][np.argmax(probs)]
    
    str_for_writer += "{}:{}-{},".format(i, pred, ans)
    
    confmat[ans][pred] += 1
    counts[ans] += 1
  
  with tf.gfile.GFile(get_name_resultfile(prefix_resultfile), "w") as writer:
    writer.write("***** Writing start *****\n")
    
    print("accuracy of global check")
    ans_g = sum([v[i] for i, v in enumerate(confmat)])
    total_g = sum([sum(v) for v in confmat])
    print("accuracy {}% ({}/{})".format(ans_g/total_g, ans_g, total_g))
    writer.write("accuracy {}% ({}/{})\n".format(ans_g/total_g, ans_g, total_g))
    
    writer.write("ans|pred\t#0\t#1\t#2\n")
    for i, v in enumerate(confmat):
      writer.write("#{}\t{}\n".format(i, "\t".join([str(n) for n in v])))

In [0]:
def make_paths_from_steps(steps=0, used=500):
  if steps is None or steps <= 0:
    checkpoint_path = None
    prefix_resultfile = "iter{}_used{}".format(0, used)
  else:
    checkpoint_path = "{}/model.ckpt-{}".format(MODEL_PATH, steps)
    prefix_resultfile = "iter{}_used{}".format(steps, used)
  
  return prefix_resultfile, checkpoint_path

estimator = make_estimator(init_checkpoint=INIT_CP, model_dir=MODEL_PATH)

prefix_resultfile, checkpoint_path = make_paths_from_steps(steps=20000, used=500)
predict(prefix_resultfile=prefix_resultfile, checkpoint_path=checkpoint_path)

W0729 11:07:27.364806 139768208836480 estimator.py:1984] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f1e04364ea0>) includes params argument, but params are not passed to Estimator.
I0729 11:07:27.366728 139768208836480 estimator.py:209] Using config: {'_model_dir': 'gs://japan-legal-term-correction/pretrain_statutory_sentence/model_after_kanjibug', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.49.96.170:8470"
    }
  }
}
isolate_session_state: true
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflo

eval tfrecord already exists. read it, with answer_file


I0729 11:07:31.651400 139768208836480 tpu_system_metadata.py:78] Querying Tensorflow master (grpc://10.49.96.170:8470) for TPU system metadata.
I0729 11:07:31.669108 139768208836480 tpu_system_metadata.py:148] Found TPU system:
I0729 11:07:31.670619 139768208836480 tpu_system_metadata.py:149] *** Num TPU Cores: 8
I0729 11:07:31.676313 139768208836480 tpu_system_metadata.py:150] *** Num TPU Workers: 1
I0729 11:07:31.677685 139768208836480 tpu_system_metadata.py:152] *** Num TPU Cores Per Worker: 8
I0729 11:07:31.681771 139768208836480 tpu_system_metadata.py:154] *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 12752772093830360347)
I0729 11:07:31.683892 139768208836480 tpu_system_metadata.py:154] *** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 8030931855453645136)
I0729 11:07:31.685997 139768208836480 tpu_system_metadata.py:154] *** Available Device: _DeviceAttributes(/job:worker/replica:0/ta

***** Started prediction at 2019-07-29 11:07:31.634102 *****
***** Finished prediction at 2019-07-29 11:07:31.634367 *****


I0729 11:07:31.993918 139768208836480 run_pretraining.py:117] *** Features ***
I0729 11:07:31.995219 139768208836480 run_pretraining.py:119]   name = input_ids, shape = (8, 128)
I0729 11:07:31.996538 139768208836480 run_pretraining.py:119]   name = input_mask, shape = (8, 128)
I0729 11:07:32.000243 139768208836480 run_pretraining.py:119]   name = masked_lm_ids, shape = (8, 20)
I0729 11:07:32.001660 139768208836480 run_pretraining.py:119]   name = masked_lm_positions, shape = (8, 20)
I0729 11:07:32.003918 139768208836480 run_pretraining.py:119]   name = masked_lm_weights, shape = (8, 20)
I0729 11:07:32.005035 139768208836480 run_pretraining.py:119]   name = next_sentence_labels, shape = (8, 1)
I0729 11:07:32.007749 139768208836480 run_pretraining.py:119]   name = segment_ids, shape = (8, 128)
I0729 11:07:36.992023 139768208836480 run_pretraining.py:167] **** Trainable Variables ****
I0729 11:07:37.984443 139768208836480 estimator.py:1147] Done calling model_fn.
I0729 11:07:37.995723 139

accuracy of global check
accuracy 0.9759384118460098% (197000/201857)


## botu

### modify pretrained japanese bert model to treat legal terms as one word

In [0]:
with tf.Session() as sess:
  saver = tf.train.Saver()
  saver.restore(JPNBERT_PATH + "/bert_model.ckpt.meta")

RuntimeError: ignored

In [0]:
%env MODEL_PATH={MODEL_PATH}
%env BERT_CONFIG_FILE={BERT_CONFIG_FILE}
%env INIT_CP={INIT_CP}

env: MODEL_PATH=gs://japan-legal-term-correction/pretrain_statutory_sentence/model
env: BERT_CONFIG_FILE=gs://japan-legal-term-correction/bert/Japanese_L-12_H-768_A-12_E-30_BPE/bert_config.json
env: INIT_CP=gs://japan-legal-term-correction/bert/Japanese_L-12_H-768_A-12_E-30_BPE/bert_model.ckpt


In [0]:
%%bash
python bert-tltc/run_pretraining.py \
--input_file=${TFRECORD_FILE} --output_dir=${MODEL_PATH} \
--do_train=True --do_eval=True \
--bert_config_file=${BERT_CONFIG_FILE} --init_checkpoint=${INIT_CP} \
--train_batch_size=32 --max_seq_length=128 --max_predictions_per_seq=20 --num_train_steps=20 --num_warmup_steps=10 --learning_rate=2e-5 2>&1

Process is terminated.
